## 获取初始文本

In [3]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import numpy as np
from tensorflow import keras

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

maxlen = 60
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Corpus length: 600893
Number of sequences: 200278
Unique characters: 57
Vectorization...


In [4]:
dict(zip(sentences, next_chars))

{'preface\n\n\nsupposing that truth is a woman--what then? is the': 'r',
 'face\n\n\nsupposing that truth is a woman--what then? is there ': 'n',
 'e\n\n\nsupposing that truth is a woman--what then? is there not': ' ',
 '\nsupposing that truth is a woman--what then? is there not gr': 'o',
 'pposing that truth is a woman--what then? is there not groun': 'd',
 'sing that truth is a woman--what then? is there not ground\nf': 'o',
 'g that truth is a woman--what then? is there not ground\nfor ': 's',
 'hat truth is a woman--what then? is there not ground\nfor sus': 'p',
 ' truth is a woman--what then? is there not ground\nfor suspec': 't',
 'uth is a woman--what then? is there not ground\nfor suspectin': 'g',
 ' is a woman--what then? is there not ground\nfor suspecting t': 'h',
 ' a woman--what then? is there not ground\nfor suspecting that': ' ',
 'woman--what then? is there not ground\nfor suspecting that al': 'l',
 'an--what then? is there not ground\nfor suspecting that all p': 'h',
 

In [5]:
# 构建模型
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from tensorflow.keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


import random
import sys

for epoch in range(60):
    print('epoch', epoch+1)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    # for temperature in [0.2, 0.5, 1.0, 1.2]:
    for temperature in [0.5]:
        print('------ temperature:', temperature)
        print(generated_text)
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)
    print()

2022-04-26 22:15:56.577229: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-04-26 22:15:56.621396: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: NVIDIA TITAN Xp computeCapability: 6.1
coreClock: 1.582GHz coreCount: 30 deviceMemorySize: 11.91GiB deviceMemoryBandwidth: 510.07GiB/s
2022-04-26 22:15:56.621472: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-04-26 22:15:56.624222: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-04-26 22:15:56.626476: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2022-04-26 22:15:56.626865: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcurand.so.10


epoch 1


2022-04-26 22:16:00.041105: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-04-26 22:16:00.284453: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7


1565/1565 [==============================] - 11s 7ms/step - loss: 1.9472
--- Generating with seed: " my eyes to schopenhauer's blind will
towards ethic, at a ti"
------ temperature: 0.5
 my eyes to schopenhauer's blind will
towards ethic, at a ti
micn and the do the predume in in any himself--a may the placious something in the fortions and hears in in the present of his to less in the deverded and are one mine as a more always and germany the deared to learness, and in all the soul dored and in the gards that we prempy the soul to the expection of the instruse of in the interrom of the pays in the like the something and prevered in the pa
epoch 2
1565/1565 [==============================] - 11s 7ms/step - loss: 1.6076
--- Generating with seed: "as they shine here and
there: those moments and marvelous ex"
------ temperature: 0.5
as they shine here and
there: those moments and marvelous ex
ceptionally to the described to lives the criration of the freedom, and classed or as a mankind o

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7f8848dc2670>
Traceback (most recent call last):
  File "/home/cfn/anaconda3/envs/zytf/lib/python3.8/weakref.py", line 345, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 


 art and allowing to be the sense of the present and contravies of the instrume the tendent in the first 
epoch 5



KeyboardInterrupt

